In [ ]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '0'
working_path = os.getcwd()

In [ ]:
import tensorflow as tf
import numpy as np
import ag.logging as log
# from PIL import Image
log.set(5)

In [ ]:
load = np.load("/pub/dataset/mario/mariokart64_dataset_0.npz")
imgs = load['images']
labels = load['labels']
dataset_examples = imgs.shape[0]
dataset_h = imgs.shape[1]
dataset_w = imgs.shape[2]
dataset_c = imgs.shape[3]
dataset_classes = labels.shape[1]
dataset_shape = imgs[0].shape
label_example = labels[0]

In [ ]:
log.info("Images: {}".format(imgs.shape))
log.info("Labels: {}".format(labels.shape))
log.info("Image shape: {}".format(dataset_shape))
log.info("Label: {}".format(label_example))

In [ ]:
class Options(): pass

log_every = 500
loss_every = 2

max_out = 100   # max concurrent losing steps before bail-out
               # note that this is only every loss_every

safety_max = int(1e5)

# Data Output Stuffs!!
save_path = '/pub/models/mupen64/mariokart64/'
all_saves = os.listdir(save_path)
num_saves = len(all_saves)
save_dirname = 'outputmodel{}'.format(num_saves)
save_filename = "mupen64_mariokart64"
new_path = os.path.join(save_path, save_dirname)
if not os.path.isdir(save_dirname):
    os.mkdir(new_path)
    print("made a new dir {}".format(new_path))
else:
    print("something..??")
save_path = os.path.join(new_path, save_filename)
print(save_path)

In [ ]:
with tf.name_scope("InitVars"):
    with tf.variable_scope("init_vars_values"):
        global_step = tf.Variable(0, trainable=False, dtype='int64')
        
        learn_rate = tf.train.exponential_decay(0.1, global_step,
                                                .05, 0.87,
                                                staircase=True, name="Learn_decay")
        
tf.add_to_collection("learn_rate", learn_rate);
tf.summary.scalar("learn_rate", learn_rate);
tf.summary.scalar("global_step", learn_rate);
tf.summary.histogram('decay_rate', learn_rate);

In [ ]:
with tf.variable_scope("inputs"):
    Input_Tensor_Image = tf.placeholder(tf.float32, [None, 
                                                     dataset_h, 
                                                     dataset_w,
                                                     dataset_c])
tf.add_to_collection("input_tensor", Input_Tensor_Image)

In [ ]:
print(Input_Tensor_Image)

In [ ]:
Input_Tensor_Labels = tf.placeholder(tf.float32, [None, dataset_classes], name="Input_Label")
tf.add_to_collection("label", Input_Tensor_Labels)
Input_True_Labels = tf.argmax(Input_Tensor_Labels, dimension=1);

In [ ]:
with tf.variable_scope("keep_prob"):
    keep_prob = tf.placeholder(tf.float32, name="Keep_prob")  # new feature goes with the dropout option
tf.add_to_collection("keep_prob", keep_prob)

In [ ]:
def BUILD_CONV_LAYERS(x_image, layers, f_size=5):
    """
    Magically construct many Convolutional layers for a Neural Network.
    :return:
        An unused list of all the layers_names, weights and biases for each layer. these are added to tensorboard
        automatically.
    """
    start_shape = x_image
    log.debug("Start shape= {}".format(start_shape))
    reducing_shape = 0
    last_num_f = 0
    pooling = False
    log.info("Building {} Convolutional Layers".format(layers))
    for layer in range(layers):
        num_f = last_num_f + (16 * (layer + 1))
        log.debug("Current Num of Features= {}".format(num_f))
        if layer == 0:
            img = start_shape
            channel = dataset_c
        else:
            img = reducing_shape
            channel = last_num_f
        
        if layer == layers -1:
            pooling = True

        last_num_f = num_f
        log.debug("Start shape= {}".format(start_shape))
        debugs = "New Features = {}".format(channel)
        reducing_shape, w, b = new_conv_layer(input=img,
                                              filter_size=f_size,
                                              chan=channel,
                                              num_filters=num_f,
                                              use_pooling=True)
        layer_name = "convLayer_%s" % layer
        log.debug("Finishing Layer {}\n\t{}".format(layer_name, debugs))
        #tf.summary.histogram("weights{}".format(layer), w)
        #tf.summary.histogram("biases{}".format(layer), b)
        log.info("Finished Layer {}\n\t{}".format(layer_name, debugs))

    log.info("Finished Building {} Conv Layers\nMoving To Flattening...".format(layers))
    return reducing_shape, last_num_f

In [ ]:
def BUILD_FC_LAYER(x_image, in_features, num_fc, num_final):
    """
    Magically create a huge number of Fully connected layers.
    :param layers:
        Number of FC layers to create.
    :return:
        returns a human readable non used list of the layer names. These are added to tensorboard automatically.
    """
    log.info("Building {} Fully Connected Layers\ninput features size = {}\n*-".format(num_fc, in_features))
    # first time thru options
    inputs = 0
    features = 0
    use_reLu = True
    use_Drop = True
    current_layer = x_image    
    for layer in range(num_fc):        
        if layer == 0:
            features = in_features
        # always set in to features and ...
        inputs = features
        # set features to a decay function of i
        features = int(inputs / int((layer + 1))) # avoid a 0 zero divion error..
        
        if layer == num_fc - 1:  # if last time through
            features = num_final
            use_reLu = False  # dont use on the last time thru
            use_Drop = False
        
        log.debug("Layer-{}\n\t# inputs: {}\n\t# outputs: {}\n\tuse relu? {}\n\tuse drop? {}".format(\
            current_layer, inputs, features, use_reLu, use_Drop))
        current_layer, w, b = new_fc_layer(input_=current_layer,
                                           num_inputs=inputs,
                                           num_outputs=features,
                                           use_relu=use_reLu,
                                           use_drop=use_Drop)
             
        layer_name = "fullyLayer_{}".format(layer)
        """
        with tf.name_scope(layer_name):
            with tf.name_scope("weights"):
                tf.summary.histogram("weights", w)
            with tf.name_scope("biases"):
                tf.summary.histogram('biases', b)
        log.info("{} : {}".format(layer_name, current_layer))
        """
    log.info("Finished Building {} Fully Connected Layers".format(num_fc))
    return current_layer # final layer

In [ ]:
def new_fc_layer(input_, num_inputs, num_outputs, use_relu=True, use_drop=False):
    weights = new_weights(shape=[num_inputs, num_outputs])  # set weights
    biases = new_biases(length=num_outputs)  # set number of OUTPUTS like give me top k or whatever...
    layer = tf.matmul(input_, weights) + biases  # this is a #BIGMATH func
    if use_relu:
        layer = tf.nn.relu(layer)
    if use_drop:
        layer = tf.nn.dropout(layer, keep_prob)
    return layer, weights, biases

In [ ]:
def new_weights(shape):
    """This generates a new weight for each layer"""
    return tf.Variable(tf.truncated_normal(shape, stddev=0.1), name="weight")

def new_biases(length):
    """This generates a new bias for each layer"""
    return tf.Variable(tf.constant(0.1, shape=[length]), name="bias")

In [ ]:
def new_conv_layer(input, filter_size, chan, num_filters, use_pooling=True):
    log.debug("Building a conv layer")
    X_shape = [filter_size, filter_size, chan, num_filters]
    log.debug("current shape: {}".format(X_shape))
    weights = new_weights(shape=X_shape)
    log.debug("weights shape = {}".format(weights))
    biases = new_biases(length=num_filters)
    log.debug("weights shape = {}".format(biases))
    """ THis is the MAGIC again... """
    layer = tf.nn.conv2d(input=input,  # This is the output of the last layer
                         filter=weights,  # this is a thing
                         strides=[1, 1, 1, 1],  # 1111 is NOT pooled MAX work
                         padding='SAME')  # input output transformation
    layer += biases  # Add biases
    if use_pooling:  # this skips pixels... saves time but skips things obviously
        log.debug("using pooling")
        layer = tf.nn.max_pool(value=layer,  # take the weights and bias together as an input
                               ksize=[1, 2, 2, 1],  # stuff
                               strides=[1, 2, 2, 1],  # 2 x 2 stride... could increase... check
                               padding='SAME')  # i feel like this should already be in variable, but w/e
    layer = tf.nn.relu(layer)  # rectified linear Unit ... like a boss
    log.debug("Finished Building a conv Layer:\n\t{}".format(layer))
    return layer, weights, biases

In [ ]:
def flatten_layer(layer, num_features):
    log.info("Tranitioning from Conv to fc layers")
    layer_shape = layer.get_shape()  # ASSERT layer_shape == [num_images, img_height, img_width, num_channels]
    num_ele = layer_shape[1:4].num_elements()  # like a boss...
    #num_ele = int(num_ele/5)
    print(num_ele)
    num_features = num_ele
    log.debug("Shape: {}, Features: {}".format(layer.shape[:], num_features))
    layer_flat = tf.reshape(layer, [-1, num_features])  # yep...
    log.info("Finished Flattening Layers")
    return layer_flat, num_features

In [ ]:
with tf.variable_scope("conv_layers"):
    x_image, num_features = BUILD_CONV_LAYERS(Input_Tensor_Image, 7)
log.info("created {} layers with {} features".format(7, num_features))

In [ ]:
with tf.variable_scope("Flat_layer"):
    x_image, features =flatten_layer(x_image, num_features)

In [ ]:
with tf.variable_scope("FC_layers"):
    final_layer = BUILD_FC_LAYER(x_image, features, num_fc=5, num_final=5)  # build FC LAYERS

In [ ]:
print(final_layer)

In [ ]:
print(Input_Tensor_Image)

In [ ]:
with tf.variable_scope("sophmax"):
    sophmax = tf.nn.softmax(final_layer, name="sophmax")
tf.add_to_collection("sophmax_layer", sophmax)
# tf.summary.histogram('activations', sophmax);

In [ ]:
### TRAINING METHOD FROM MUPEN #### MEPHOD?
with tf.variable_scope("mupen_method"):
    cost = tf.reduce_mean(tf.square(tf.subtract(Input_Tensor_Labels, final_layer)))
    train_vars = tf.trainable_variables()
#    loss = cost +\
#			tf.add_n([tf.nn.l2_loss(v) for v in train_vars]) *\
#			0.001
    summ = []
    for v in train_vars:
        log.debug("v: {}".format(v))
        summ.append(tf.nn.l2_loss(v))
    loss = cost + tf.add_n(summ) * 0.001
    # loss = cost + training_vars
tf.add_to_collection("loss", loss)
tf.summary.scalar("train_cost", loss)

In [ ]:
with tf.variable_scope('Dropout_Optimizer_Train'):
    #train_drop_loss = tf.train.AdagradOptimizer(learn_rate).minimize(loss)
    #train_drop_loss = tf.train.AdamOptimizer(learn_rate).minimize(loss)
    #train_drop_loss = tf.train.GradientDescentOptimizer(learn_rate).minimize(loss)
    #train_drop_loss = tf.train.FtrlOptimizer(learn_rate).minimize(loss)
    #train_drop_loss = tf.train.MomentumOptimizer(learn_rate, momentum=5).minimize(loss)
    
tf.add_to_collection("train_op", train_drop_loss);

In [ ]:
### OHHHH WE ARE GETTING CLOSE! ###
""" Initialize the session """
init_op = tf.global_variables_initializer

"""create summary op"""
merged = tf.summary.merge_all

"""create saver object"""
saver = tf.train.Saver(
#            #var_list={"{}".format(v): v for v in [tf.model_variables()]},
#            write_version=tf.train.SaverDef.V2,
#            sharded=True,
#            keep_checkpoint_every_n_hours=.001
            )

In [ ]:
# Training loop variables
epochs = 2
batch_size = 50
num_samples = imgs.shape[0]
step_size = int(num_samples / batch_size) 
log.info("Training Run Options:\n\tTraining data resets: {}\n\tBatch size: {}\n\tStep Size: {}".format(\
            epochs, batch_size, step_size))

In [ ]:
# Single use for demonstration Session...
# Start session
#sess = tf.InteractiveSession("grpc://localhost:2222")
sess = tf.InteractiveSession()
log.info("Started Session: {}".format(sess))
sess.run(tf.global_variables_initializer())
log.info("started Init Op")
log.debug("starting train writer: {}".format(save_path))
train_writer = tf.summary.FileWriter(save_path, sess.graph)
log.info("train writer started")

In [ ]:
class data_prep(object):
    def __init__(self, images, labels):
        self.index_in_epoch = 0
        self.num_examples = 0 
        self.epochs_completed = 0
        self.num_examples = 0
        self.images = images
        self.labels = labels
        self.check_data()
        
    def check_data(self):
        try:
            assert self.images.shape[0] == self.labels.shape[0]
        except Exception as e:
            print("{} is not {}".format(self.images.shape[0], self.labels.shape[0]))
        self.num_examples = self.images.shape[0]
        return True
        

    def next_batch(self, batch_size, shuffle=False):
            """ Shuffle is off by default """
            start = self.index_in_epoch
            self.index_in_epoch += batch_size
            if self.index_in_epoch > self.num_examples:
                # Finished epoch                                                                                       
                self.epochs_completed += 1
                # Shuffle the data                                                                                     
                if shuffle:
                    perm = np.arange(self.num_examples) # should add some sort of seeding for verification            
                    np.random.shuffle(perm)
                    self.images = self.images[perm]
                    self.labels = self.labels[perm]
                # Start next epoch                                                                                     
                start = 0
                self.index_in_epoch = batch_size
                assert batch_size <= self.num_examples
            end = self.index_in_epoch
            return self.images[start:end], self.labels[start:end]

In [ ]:
batcher = data_prep(imgs, labels)
print(batcher.images.shape[:])
test_batch = batcher.next_batch(50)
print(test_batch[0].shape[:])

In [ ]:
### Training Proceedures
current_epoch = 0
# while current_epoch < epochs:
gstep = 0
best = 1e100 # best score to beat (loss: lower is better)
losing_streak = 0

for step in range(safety_max):

    # get a batch
    batch = batcher.next_batch(42)

    # common feed dict
    feed_dict = {Input_Tensor_Image: batch[0],
                 Input_Tensor_Labels: batch[1],
                 keep_prob: 0.8}

    # be verbose...
    # do the step.
    
    #log.debug("train drop loss")
    _ = sess.run(train_drop_loss, feed_dict=feed_dict)
    #log.info("train loss result: {}".format(_))
    
    #log.debug("global_step")
    gstep = sess.run(global_step, feed_dict=feed_dict)
    log.info("global step: {}".format(gstep))
    
    #log.debug("learn_rate")
    learn = sess.run(learn_rate, feed_dict=feed_dict)
    #log.info("learn rate is: {}".format(learn))
    
    if step % loss_every == 0:
        #log.debug("cost")
        new_loss = sess.run(loss, feed_dict=feed_dict)
        log.info("Loss: {}".format(new_loss))

        if (new_loss < best):
            best = new_loss
            losing_streak = 0
            log.info("# - A new High Score has been reached: {}".format(best))
            
            log.info("saving...")
            saver.save(sess, save_path, global_step=gstep)
            
        else: # didn't beat high score
            log.debug("   losing... {}".format(losing_streak))
            losing_streak += 1
            
            if losing_streak >= max_out:
                log.info("bailing out for no recent progress!")
                break
    
    if step % log_every == 0:    
        log.debug("merging")
        #log.debug("merged is: {}".format(merged))
        summary = sess.run(merged(), feed_dict=feed_dict)
        log.debug("summary size: {}".format(len(summary)))
        
        log.debug("adding to tensorboard summary")
        train_writer.add_summary(summary, step)


    global_step += 1

log.debug("final save")
saver.save(sess, save_path, global_step=gstep)

print("finished training")